In [1]:
import xarray as xr
import rioxarray
import rasterio
import numpy as np
import dask
from dask.distributed import Client
#client = Client()
#client
dask.config.set(scheduler="synchronous")

In [2]:
import os
if 'notebooks' in os.getcwd():
    os.chdir("..")
    import sys; sys.path.append(os.getcwd())

CHUNK_SCHEMA = dict(time=27, x=512, y=512)

Read in all the data types

In [18]:
def preprocess_mortality(ds):
    year = os.path.basename(ds.encoding["source"])
    year = int(year.replace(".tif", ""))
    return ds.squeeze(drop=True)\
        .drop_vars("spatial_ref")\
        .rename(band_data="mortality")\
        .assign_coords(time=year)

def preprocess_fam(ds):
    year = os.path.basename(ds.encoding["source"])
    year = int(year.replace(".tif", ""))
    return ds.squeeze(drop=True)\
        .drop_vars("spatial_ref")\
        .rename(band_data="fam")\
        .assign_coords(time=year)

damage = xr.open_mfdataset(
    "data_working/damage_rasters/*.tif",
    preprocess=preprocess_mortality,
    concat_dim="time", 
    combine="nested"
).chunk(**CHUNK_SCHEMA)

fam = xr.open_mfdataset(
    "data_working/fam_rasters/*.tif",
    preprocess=preprocess_fam,
    concat_dim="time", 
    combine="nested"
).chunk(**CHUNK_SCHEMA)

In [19]:
damage

<xarray.Dataset> Size: 665MB
Dimensions:    (time: 27, y: 2606, x: 2361)
Coordinates:
  * x          (x) float64 19kB -1.39e+07 -1.389e+07 ... -1.154e+07 -1.154e+07
  * y          (y) float64 21kB 6.28e+06 6.279e+06 ... 3.676e+06 3.675e+06
  * time       (time) int64 216B 1997 1998 1999 2000 ... 2020 2021 2022 2023
Data variables:
    mortality  (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>

In [20]:
fam

<xarray.Dataset> Size: 665MB
Dimensions:  (time: 27, y: 2606, x: 2361)
Coordinates:
  * x        (x) float64 19kB -1.39e+07 -1.389e+07 ... -1.154e+07 -1.154e+07
  * y        (y) float64 21kB 6.28e+06 6.279e+06 ... 3.676e+06 3.675e+06
  * time     (time) int64 216B 1997 1998 1999 2000 2001 ... 2020 2021 2022 2023
Data variables:
    fam      (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>

In [5]:
daymet = xr.open_mfdataset('data_working/daymet/*.nc', concat_dim="time", combine="nested")\
    .drop_vars("spatial_ref")\
    .chunk(**CHUNK_SCHEMA)
daymet

<xarray.Dataset> Size: 2GB
Dimensions:  (time: 27, y: 2606, x: 2361)
Coordinates:
  * time     (time) int64 216B 1997 1998 1999 2000 2001 ... 2020 2021 2022 2023
  * x        (x) float64 19kB -1.39e+07 -1.389e+07 ... -1.154e+07 -1.154e+07
  * y        (y) float64 21kB 6.28e+06 6.279e+06 ... 3.676e+06 3.675e+06
Data variables:
    prcp     (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    vp       (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    tmin     (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>

In [6]:
def preprocess_fire(ds):
    year = os.path.basename(ds.encoding["source"])
    year = int(year.replace(".tif", ""))
    return ds.squeeze(drop=True)\
        .drop_vars("spatial_ref")\
        .rename(band_data="fire")\
        .assign_coords(time=year)

fire = xr.open_mfdataset(
    "data_working/mtbs_rasters/*.tif",
    preprocess=preprocess_fire,
    concat_dim="time", combine="nested"
).chunk(**CHUNK_SCHEMA)

In [7]:
fire

<xarray.Dataset> Size: 665MB
Dimensions:  (time: 27, y: 2606, x: 2361)
Coordinates:
  * x        (x) float64 19kB -1.39e+07 -1.389e+07 ... -1.154e+07 -1.154e+07
  * y        (y) float64 21kB 6.28e+06 6.279e+06 ... 3.676e+06 3.675e+06
  * time     (time) int64 216B 1997 1998 1999 2000 2001 ... 2020 2021 2022 2023
Data variables:
    fire     (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>

In [10]:
def preprocess_spp_ba(ds):
    spp = os.path.basename(ds.encoding["source"])\
        .replace(".tif", "")

    return ds.squeeze(drop=True).rename(band_data=spp)

spp_ba = xr.open_mfdataset(
    "data_working/genus_basal_area/*.tif", 
    preprocess=preprocess_spp_ba,
    combine="nested"
).drop_vars("spatial_ref").expand_dims(time=damage.time.shape[0]).assign_coords(time=damage.time).chunk(**CHUNK_SCHEMA)

In [11]:
spp_ba

<xarray.Dataset> Size: 3GB
Dimensions:      (x: 2361, y: 2606, time: 27)
Coordinates:
  * x            (x) float64 19kB -1.39e+07 -1.389e+07 ... -1.154e+07 -1.154e+07
  * y            (y) float64 21kB 6.28e+06 6.279e+06 ... 3.676e+06 3.675e+06
  * time         (time) int64 216B 1997 1998 1999 2000 ... 2020 2021 2022 2023
Data variables:
    abies        (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    picea        (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    populus      (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    pseudotsuga  (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    tsuga        (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>

In [12]:
terrain = xr.open_dataset("data_working/terrain.nc")\
    .drop_vars(["spatial_ref", "band"])\
    .expand_dims(time=damage.time.shape[0])\
    .assign_coords(time=damage.time)\
    .chunk(**CHUNK_SCHEMA)

terrain

<xarray.Dataset> Size: 3GB
Dimensions:    (x: 2361, y: 2606, time: 27)
Coordinates:
  * x          (x) float64 19kB -1.39e+07 -1.389e+07 ... -1.154e+07 -1.154e+07
  * y          (y) float64 21kB 6.28e+06 6.279e+06 ... 3.676e+06 3.675e+06
  * time       (time) int64 216B 1997 1998 1999 2000 ... 2020 2021 2022 2023
Data variables:
    elev       (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    slope      (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    northness  (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    eastness   (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>

In [13]:
vod = xr.open_dataset("data_working/summer_vod.nc")\
    .drop_vars("spatial_ref")\
    .chunk(**CHUNK_SCHEMA)
vod = vod.assign_coords(time=vod.time.dt.year)
vod

<xarray.Dataset> Size: 1GB
Dimensions:    (x: 2361, y: 2606, time: 21)
Coordinates:
  * x          (x) float64 19kB -1.39e+07 -1.389e+07 ... -1.154e+07 -1.154e+07
  * y          (y) float64 21kB 6.28e+06 6.279e+06 ... 3.676e+06 3.675e+06
  * time       (time) int64 168B 1998 1999 2000 2001 ... 2015 2016 2017 2018
Data variables:
    vod        (time, y, x) float32 517MB dask.array<chunksize=(21, 512, 512), meta=np.ndarray>
    delta_vod  (time, y, x) float32 517MB dask.array<chunksize=(21, 512, 512), meta=np.ndarray>
Attributes: (12/29)
    calendar:                  standard
    geospatialverticalmin:     0.0
    timecoverageresolution:    P1D
    DOI:                       10.5281/zenodo.2575599
    product_version:           01.0
    geospatialverticalmax:     0.0
    ...                        ...
    creatorurl:                http://rs.geo.tuwien.ac.at
    license:                   Creative Commons Attribution 4.0 International
    creatorname:               Department of Geodesy and Geoinformation, Tech...
    timecoveragestartdate:     1997-12-01T00:00:00
    contact:                   vodca@geo.tuwien.ac.at
    spatialresolution:         25 km

Merge everybody together. Note this is very fast because all the arrays are on dask and have the same chunking strategy.

In [24]:
all_data = xr.combine_by_coords(
    [vod, terrain, spp_ba, fire, daymet, damage, fam],
    coords="minimal",
    compat="override",
    combine_attrs="drop"
).drop_vars(["delta_vod"])

In [25]:
all_data

<xarray.Dataset> Size: 11GB
Dimensions:      (x: 2361, y: 2606, time: 27)
Coordinates:
  * x            (x) float64 19kB -1.39e+07 -1.389e+07 ... -1.154e+07 -1.154e+07
  * y            (y) float64 21kB 6.28e+06 6.279e+06 ... 3.676e+06 3.675e+06
  * time         (time) int64 216B 1997 1998 1999 2000 ... 2020 2021 2022 2023
Data variables: (12/16)
    abies        (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    picea        (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    populus      (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    pseudotsuga  (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    tsuga        (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    vod          (time, y, x) float32 664MB dask.array<chunksize=(21, 512, 512), meta=np.ndarray>
    ...           ...
    fam          (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    fire         (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    mortality    (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    prcp         (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    vp           (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>
    tmin         (time, y, x) float32 664MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>

In [29]:
int_encoding  = {"dtype": "int16", "_FillValue": -9999}
byte_encoding = {"dtype": "int8", "_FillValue": -128}

all_data.to_netcdf(
    "data_working/alldata.nc",
    encoding={
        "abies": int_encoding,
        "picea": int_encoding,
        "populus": int_encoding,
        "pseudotsuga": int_encoding,
        "tsuga": int_encoding,
        "vod": int_encoding,
        "elev": int_encoding,
        "slope": int_encoding,
        "northness": int_encoding,
        "eastness": int_encoding,
        "fire": byte_encoding,
        "mortality": byte_encoding,
        "fam": byte_encoding,
        "prcp": int_encoding,
        "vp": int_encoding,
        "tmin": byte_encoding
    }
)

In [28]:
daymet.tmin.isel(time=0).min().compute()

<xarray.DataArray 'tmin' ()> Size: 4B
array(-25., dtype=float32)
Coordinates:
    time     int64 8B 1997

In [ ]:
# Shift so we are predicting *next year's* mortality in each sample
all_data["mortality"] = all_data["mortality"].shift(time=-1)

all_data = all_data.sel(time=slice(2000, 2017))

In [13]:
chunk = all_data.isel(x=slice(1024, 1536), y=slice(1024, 1536)).compute()

Construct a stacked dataset, rolling will happen just before write so we can do all the windows at once.

In [14]:
all_stack = all_data.to_stacked_array(new_dim="band", sample_dims=["time", "x", "y"])

In [15]:
all_stack

<xarray.DataArray 'abies' (time: 18, y: 3251, x: 2741, band: 16)> Size: 10GB
dask.array<concatenate, shape=(18, 3251, 2741, 16), dtype=float32, chunksize=(18, 512, 512, 1), chunktype=numpy.ndarray>
Coordinates:
  * x         (x) float64 22kB -1.424e+07 -1.424e+07 ... -1.15e+07 -1.15e+07
  * y         (y) float64 26kB 6.674e+06 6.673e+06 ... 3.425e+06 3.424e+06
  * time      (time) int64 144B 2000 2001 2002 2003 2004 ... 2014 2015 2016 2017
  * band      (band) object 128B MultiIndex
  * variable  (band) <U11 704B 'abies' 'picea' 'pinus' ... 'prcp' 'vp' 'tmin'

Write data as zarr arrays.

In [16]:
def enumerate_xy_chunks(array):
    steps_x = array.chunks[2]
    steps_y = array.chunks[1]

    upto_x = 0
    for sx in steps_x:
        upto_y = 0
        for sy in steps_y:
            chunk = array.isel(
                x=slice(upto_x, upto_x + sx),
                y=slice(upto_y, upto_y + sy)
            )
            upto_y += sy
            yield chunk
            
        upto_x += sx

In [17]:
damage

<xarray.Dataset> Size: 962MB
Dimensions:    (time: 27, y: 3251, x: 2741)
Coordinates:
  * x          (x) float64 22kB -1.424e+07 -1.424e+07 ... -1.15e+07 -1.15e+07
  * y          (y) float64 26kB 6.674e+06 6.673e+06 ... 3.425e+06 3.424e+06
  * time       (time) int64 216B 1997 1998 1999 2000 ... 2020 2021 2022 2023
Data variables:
    mortality  (time, y, x) float32 962MB dask.array<chunksize=(27, 512, 512), meta=np.ndarray>

In [17]:
# 2-10 years. Based on initial experiment with CA dataset, the temporal
# benefit plateaus around 5 years. Could be better with this larger
# dataset so we will test a range of values.
import zarr
from google.cloud import storage
from pathlib import Path

client = storage.Client(project="forest-lst")
bucket = client.bucket("ads_training_data")

time_windows = list(range(2, 11))

def write_windowed_data(chunk):
    for tw in time_windows:
        print(f"{tw}...", end="")
        zarr_chunk_size = (200_000, 16, tw)
        # print("Now writing", tw)
        out = f"gs://ads_training_data/windowed{tw}.zarr"
        
        #print("Windowing...")
        chunk_rolling = chunk.rolling(time=tw).construct(time="time_window")\
            .stack(sample=["time", "x", "y"])\
            .transpose("sample", ...)

        #print("Slicing non-na samples...")
        nona_idx = np.where(chunk_rolling.notnull().all(dim=["band", "time_window"]))[0]
        chunk_nona = chunk_rolling.isel(sample=nona_idx)
        #print("No-na shape", chunk_nona.values.shape)
        # Since the time window is increasing, if there are no samples in this
        # window, we can safely skip all remaining windows.
        if not chunk_nona.values.shape[0]:
            print("all-NA", end="")
            break

        #print("Writing to store...")
        if storage.Blob(bucket=bucket, name=f"windowed{tw}.zarr/.zarray").exists(client):
            # If the store already exists, append to the first axis.
            store = zarr.creation.open_array(out)
            store.append(chunk_nona.values, axis=0)
        else:
            # Otherwise, make a store that has the shape of the first
            # chunk we are writing.
            store = zarr.creation.open_array(
                out, chunks=zarr_chunk_size, shape=chunk_nona.values.shape,
                cache_metadata=False, cache_attrs=False, mode="w" 
            )
            store[::] = chunk_nona.values

In [19]:
def write_single_pixel_years(chunk, prop_train=0.70, prop_valid=0.20, prop_test=0.10):
    zarr_chunk_size = (200_000, 16)
    out = "gs://ads_training_data/single_pixel_year"

    # Identify non-na pixel-years
    chunk_stack = chunk.stack(sample=["time", "x", "y"]).transpose("sample", ...)
    nona_idx = np.where(chunk_stack.notnull().all(dim="band"))[0]
    chunk_nona = chunk_stack.isel(sample=nona_idx)

    if not chunk_nona.values.shape[0]:
        # print("all-NA", end="")
        return

    # Shuffle and do train/valid/test split
    np.random.shuffle(chunk_nona.values)

    idx_train = int(chunk_nona.values.shape[0] * prop_train)
    idx_valid = int(chunk_nona.values.shape[0] * (prop_train+prop_valid))

    chunk_train, chunk_valid, chunk_test = np.split(chunk_nona.values, [idx_train, idx_valid])

    # Write
    # Assuming here that if training exists, then test/valid do as well.
    arr_exists = storage.Blob(bucket=bucket, name="single_pixel_year/training.zarr/.zattrs").exists(client)
    
    for store, arr in zip(["training.zarr", "valid.zarr", "test.zarr"], [chunk_train, chunk_valid, chunk_test]):
        if arr_exists:
            # If the store already exists, append to the sample axis and shift coordinates forward
            # print("appending to existing store")
            append_dim = "sample"
            n_existing_samples = zarr.open(out + "/" + store)["pixel"].shape[0]
            encoding = None
        else:
            # print("making new store")
            # Specify encoding so that chunk size is respected
            append_dim = None
            n_existing_samples = 0
            encoding = {"pixel": {"chunks": zarr_chunk_size}}

        # print(append_dim, n_existing_samples, encoding)
        chk_ds = xr.Dataset(
            data_vars=dict(pixel=(["sample", "band"], arr)),
            coords=dict(sample=np.arange(n_existing_samples, n_existing_samples+arr.shape[0]))
        ).chunk(sample=zarr_chunk_size[0])
        # print(out + "/" + store, append_dim, n_existing_samples)
        chk_ds.to_zarr(
            out + "/" + store, append_dim=append_dim, 
            mode="a-", encoding=encoding, safe_chunks=False
        )
        

In [ ]:
from tqdm import tqdm

for chunk in tqdm(enumerate_xy_chunks(all_stack), total=len(all_stack.chunks[1]) * len(all_stack.chunks[2])):
    chunk = chunk.compute()
    write_single_pixel_years(chunk)

In [18]:
samples_per_chunk = 200_000
out = "gs://ads_training_data/damage_xyt"
sync = zarr.sync.ThreadSynchronizer()

def write_damage_window(
    chunk, stride_x=5, stride_y=5, stride_t=5,
    prop_train=0.70, prop_valid=0.20, prop_test=0.10
):
    # Window
    chunk_rolling = chunk.rolling(x=stride_x, y=stride_y, time=stride_t)\
        .construct(x="window_x", y="window_y", time="window_t")\
        .stack(sample=["time", "x", "y"])\
        .reset_index("sample")\
        .transpose("sample", ...)

    # Drop NAs
    nona_idx = np.where(chunk_rolling.notnull().all(dim=["window_x", "window_y", "window_t"]))[0]
    chunk_nona = chunk_rolling.isel(sample=nona_idx)

    # Bail out if chunk is fully NA
    if not chunk_nona.values.shape[0]:
        return

    # Shuffle
    np.random.shuffle(chunk_nona.values)

    # Split
    idx_train = int(chunk_nona.values.shape[0] * prop_train)
    idx_valid = int(chunk_nona.values.shape[0] * (prop_train+prop_valid))

    chunk_train, chunk_valid, chunk_test = np.split(chunk_nona.values, [idx_train, idx_valid])

    # Write
    # Assuming here that if training exists, then test/valid do as well.
    arr_exists = storage.Blob(bucket=bucket, name="damage_xyt/training.zarr/.zattrs").exists(client)
    
    for store, arr in zip(["training.zarr", "valid.zarr", "test.zarr"], [chunk_train, chunk_valid, chunk_test]):
        if arr_exists:
            # If the store already exists, append to the sample axis and shift coordinates forward
            append_dim = "sample"
            n_existing_samples = zarr.open(out + "/" + store)["damage"].shape[0]
            encoding = None
        else:
            # Specify encoding so that chunk size is respected
            append_dim = None
            n_existing_samples = 0
            encoding = {"damage": {"chunks": (samples_per_chunk, 5, 5, 5)}}
            
        chk_ds = xr.Dataset(
            data_vars=dict(damage=(["sample", "window_x", "window_y", "window_t"], arr)),
            coords=dict(sample=np.arange(n_existing_samples, n_existing_samples+arr.shape[0]))
        )
        # print(out + "/" + store, append_dim, n_existing_samples)
        chk_ds.chunk(sample=samples_per_chunk).to_zarr(
            out + "/" + store, append_dim=append_dim, 
            mode="a-", encoding=encoding)

In [ ]:
from tqdm import tqdm

for chunk in tqdm(enumerate_xy_chunks(all_data["mortality"]), total=len(all_stack.chunks[1]) * len(all_stack.chunks[2])):
    chunk = chunk.compute()
    write_damage_window(chunk)